In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import langevin
from scipy.stats import pearsonr
from scipy.optimize import root

SMALL_SIZE = 16
MEDIUM_SIZE = 18
BIGGER_SIZE = 20

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

#SEED = 35010732 # from random.org
#np.random.seed(SEED)

print(plt.style.available)
plt.style.use('seaborn-white')

['seaborn-dark', 'seaborn-darkgrid', 'seaborn-ticks', 'fivethirtyeight', 'seaborn-whitegrid', 'classic', '_classic_test', 'fast', 'seaborn-talk', 'seaborn-dark-palette', 'seaborn-bright', 'seaborn-pastel', 'grayscale', 'seaborn-notebook', 'ggplot', 'seaborn-colorblind', 'seaborn-muted', 'seaborn', 'Solarize_Light2', 'seaborn-paper', 'bmh', 'tableau-colorblind10', 'seaborn-white', 'dark_background', 'seaborn-poster', 'seaborn-deep']


In [2]:
def correlated_ts(c,delta_t = 0.1,N=1000):
    # parameters for coupled oscillator
    K,D = 1.0,1.0
    data1 = langevin.time_series(A=1/K, D=D, delta_t=delta_t, N=N)
    data2 = langevin.time_series(A=1/(K+np.abs(c)), D=D, delta_t=delta_t, N=N)
    x1 = (data1 + data2)/2
    if c>0:
        x2 = (data1 - data2)/2
    else:
        x2 = (data2-data1)/2

    return x1,x2

def c_rho(rho):
    return 2*np.abs(rho)/(1-np.abs(rho))*np.sign(rho)


In [3]:
def calc_fundstats(x):
    return x[0]**2+x[-1]**2,np.sum(x[1:-1]**2),np.sum(x[0:-1]*x[1:])

In [4]:
def b(D,A,delta_t):
    return np.exp(-D/A*delta_t)

def q(aep,ass,ac,b):
    return (aep + (1+b**2)*ass - 2*b*ac)/(1-b**2)

def dqdB(aep,ass,ac,b):
    return 2*(b*aep+2*b*ass-(1+b**2)*ac)/(1-b**2)**2

def d2qdB2(aep,ass,ac,b):
    return (6*b+2)/(1-b**2)**3*(aep+2*ass)-(4*b**3+12*b)/(1-b**2)**3*ac

def dBdA(b,D,A,delta_t):
    return b*D*delta_t/A**2

def dBdD(b,A,delta_t):
    return -b*delta_t/A

def d2BdA2(b,D,A,delta_t):
    return b*D*delta_t/A**3*(D*delta_t/A-2)

def d2BdD2(b,A,delta_t):
    return b*delta_t**2/A**2

def d2BdAdD(b,D,A,delta_t):
    return b*delta_t/A**2*(1-D*delta_t/A)

def d2qdD2(aep,ass,ac,b,A,delta_t):
    return d2qdB2(aep,ass,ac,b)*dBdD(b,A,delta_t)**2+dqdB(aep,ass,ac,b)*d2BdD2(b,A,delta_t)

def d2qdA2(aep,ass,ac,b,D,A,delta_t):
    return d2qdB2(aep,ass,ac,b)*dBdA(b,D,A,delta_t)**2+dqdB(aep,ass,ac,b)*d2BdA2(b,D,A,delta_t)

def d2qdAdD(aep,ass,ac,b,D,A,delta_t):
    return d2qdB2(aep,ass,ac,b)*dBdA(b,D,A,delta_t)*dBdD(b,A,delta_t)+dqdB(aep,ass,ac,b)*d2BdAdD(b,D,A,delta_t)

def d2PdA2(N,aep,ass,ac,b,D,A,delta_t):
    return (N/2/A**2 - 
            q(aep,ass,ac,b)/A**3 +
            (N-1)/(1-b**2)*(b*d2BdA2(b,D,A,delta_t) + dBdA(b,D,A,delta_t)**2*(1+b**2)/(1-b**2)) -
            d2qdA2(aep,ass,ac,b,D,A,delta_t)/2/A +
           1/A*dqdB(aep,ass,ac,b)*dBdA(b,D,A,delta_t))
        
def d2PdAdD(N,aep,ass,ac,b,D,A,delta_t):
    return (dqdB(aep,ass,ac,b)*dBdD(b,A,delta_t)/2/A**2 -
            d2qdAdD(aep,ass,ac,b,D,A,delta_t)/2/A +
            (N-1)/(1-b**2)*(b*d2BdAdD(b,D,A,delta_t) + dBdA(b,D,A,delta_t)*dBdD(b,A,delta_t)*(1+b**2)/(1-b**2)))

def d2PdD2(N,a1ep,a1ss,a1c,a2ep,a2ss,a2c,b1,b2,D,A1,A2,delta_t):
    return ((N-1)/(1-b1**2)*(b1*d2BdD2(b1,A1,delta_t) + dBdD(b1,A1,delta_t)**2*(1+b1**2)/(1-b1**2))+
           (N-1)/(1-b2**2)*(b2*d2BdD2(b2,A2,delta_t) + dBdD(b2,A2,delta_t)**2*(1+b2**2)/(1-b2**2))-
           d2qdD2(a1ep,a1ss,a1c,b1,A1,delta_t)/2/A1 -
           d2qdD2(a2ep,a2ss,a2c,b2,A2,delta_t)/2/A2)
           
def phi_deriv(x,a1ep,a1ss,a1c,a2ep,a2ss,a2c,delta_t,N):
    # x[0] = A1, x[1] = A2, x[2]=D
    A1 = x[0]
    A2 = x[1]
    D = x[2]
    b1 = b(D,A1,delta_t)
    b2 = b(D,A2,delta_t)
    Q1 = q(a1ep,a1ss,a1c,b1)
    Q2 = q(a2ep,a2ss,a2c,b2)
    dQ1 = dqdB(a1ep,a1ss,a1c,b1)
    dQ2 = dqdB(a2ep,a2ss,a2c,b2)
    y1 = -N*A1**2/2 + A1*Q1/2 + b1*D*delta_t*(A1*b1*(N-1)/(1-b1**2)-dQ1/2)
    y2 = -N*A2**2/2 + A2*Q2/2 + b2*D*delta_t*(A2*b2*(N-1)/(1-b2**2)-dQ2/2)
    y3 = (b1*(N-1)/(1-b1**2)-dQ1/A1/2)*b1/A1 + (b2*(N-1)/(1-b2**2)-dQ2/A2/2)*b2/A2
    return np.array([y1,y2,y3])

In [5]:
def d2PdA2N(N,b,A,delta_t):
    return -N/2/A - N/(1-b**2)**2*dBdA(b,D,A,delta_t)**2*(1+b**2+6*b/(1+b))+2*N*b/(1-b**2)*dBdA(b,D,A,delta_t)

def d2PdAdDN(N,b,D,A,delta_t):
    return N*b/(1-b**2)*dBdD(b,A,delta_t) - N/(1-b**2)**2*dBdA(b,D,A,delta_t)*dBdD(b,A,delta_t)*(1+b**2+6*b/(1+b))

def d2PdD2N(N,b1,b2,D,A1,A2,delta_t):
    return (-N/(1-b1**2)**2*dBdD(b1,A1,delta_t)**2*(1+b1**2+6*b1/(1+b1))-
            N/(1-b2**2)**2*dBdD(b2,A2,delta_t)**2*(1+b2**2+6*b2/(1+b2)))

In [7]:
corr1k05 = pd.read_csv("correlations1k05.csv")
corr1k05

,rho,coupling,pearson,a1,da1,a2,da2,d,dd,a1ep,a1ss,a1c,a2ep,a2ss,a2c,c,dc
0,0.5,2.0,0.429276,0.900333,0.077475,0.356596,0.020181,0.932768,0.038209,1.374045,882.989843,643.194441,1.075522,352.641817,160.590216,1.533115,0.263663
1,0.5,2.0,0.475215,0.983414,0.083328,0.347273,0.018769,0.993904,0.042320,1.093765,965.929076,707.357973,1.036471,342.762308,146.734498,1.839862,0.283390
2,0.5,2.0,0.501532,1.041411,0.091089,0.335957,0.017564,1.025705,0.042548,1.451544,1022.965428,750.157506,1.241180,329.481648,138.822811,2.108188,0.315812
3,0.5,2.0,0.476623,0.962169,0.081413,0.332260,0.017791,0.983599,0.040760,1.229173,940.497825,700.131122,1.296571,332.478514,126.485851,1.904124,0.291207
4,0.5,2.0,0.431407,0.899511,0.072164,0.354984,0.019200,1.024875,0.043741,2.397144,881.117396,623.575061,1.541302,351.129210,145.857448,1.540962,0.242017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0.5,2.0,0.469117,0.965383,0.082081,0.343594,0.019369,0.961891,0.040275,3.816385,939.497428,691.406683,1.464865,339.391459,146.063356,1.818704,0.289306
496,0.5,2.0,0.481844,0.922816,0.078574,0.319107,0.017319,0.964434,0.039848,1.082935,903.805932,660.039271,3.138795,313.931326,123.285160,1.900324,0.292762
497,0.5,2.0,0.514214,1.040725,0.090418,0.316965,0.016064,1.022044,0.042236,1.210762,1019.389176,760.305016,1.198667,314.783016,115.832788,2.291765,0.330679
498,0.5,2.0,0.445304,1.053270,0.093172,0.393661,0.021849,0.969351,0.039463,4.406811,1021.254257,774.555996,1.054270,391.301891,179.743602,1.683665,0.279009


In [18]:
rho = 0.5
delta_t = 0.3
N = 1000
c = 2*rho/(1-rho)
guessa1 = 1.0
guessa2 = 1.0/(1.0+c)
guessd = 1.0
A1_list = []
A2_list = []
dA1_list = []
dA2_list = []
C_list = []
dC_list = []
D_list = []
dD_list = []
tau1_list = []
dtau1_list = []
tau2_list = []
dtau2_list = []
print(guessa1,guessa2,guessd,c)
for index, row in corr1k05.iterrows():
    a1ep,a1ss,a1c = row['a1ep'], row['a1ss'], row['a1c']
    a2ep,a2ss,a2c = row['a2ep'], row['a2ss'], row['a2c']
    para = (a1ep,a1ss,a1c,a2ep,a2ss,a2c,delta_t,N)
    result = root(phi_deriv, [guessa1,guessa2,guessd],args=para)
    A1 = result.x[0]
    A2 = result.x[1]
    D = result.x[2]
    # lets calculate the Hessian
    b1 = b(D,A1,delta_t)
    b2 = b(D,A2,delta_t)
    d2PdA2_1m = d2PdA2(N,a1ep,a1ss,a1c,b1,D,A1,delta_t)
    d2PdA2_2m = d2PdA2(N,a2ep,a2ss,a2c,b2,D,A2,delta_t)
    d2PdD2m = d2PdD2(N,a1ep,a1ss,a1c,a2ep,a2ss,a2c,b1,b2,D,A1,A2,delta_t)
    d2PdAdD_1m = d2PdAdD(N,a1ep,a1ss,a1c,b1,D,A1,delta_t)
    d2PdAdD_2m = d2PdAdD(N,a2ep,a2ss,a2c,b2,D,A2,delta_t)
    hessian = np.array([[d2PdA2_1m,0,d2PdAdD_1m],[0,d2PdA2_2m,d2PdAdD_2m],[d2PdAdD_1m,d2PdAdD_2m,d2PdD2m]])
    var = -np.linalg.inv(hessian)
    dA1 = np.sqrt(var[0,0])
    dA2 = np.sqrt(var[1,1])
    dD = np.sqrt(var[2,2])
    dA1A2 = var[0,1]
    dA1dD = var[0,2]
    dA2dD = var[1,2]
    C = (A1-A2)/A2
    dC = np.sqrt(1/A2**2*dA1**2+A1**2/A2**4*dA2**2-A1/A2**4*dA1A2)
    tau1 = A1/D
    tau2 = A2/D
    dtau1 = np.sqrt(1/D**2*dA1+A1**2/D**4*dD-A1/D**3*dA1dD)
    dtau2 = np.sqrt(1/D**2*dA1+A1**2/D**4*dD-A1/D**3*dA2dD)
    print(A1,dA1,A2,dA2,D,dD,C,dC)
    # add results to list
    A1_list.append(A1)
    A2_list.append(A2)
    dA1_list.append(dA1)
    dA2_list.append(dA2)
    D_list.append(D)
    dD_list.append(dD)
    C_list.append(C)
    dC_list.append(dC)
    tau1_list.append(tau1)
    tau2_list.append(tau2)
    dtau1_list.append(dtau1)
    dtau2_list.append(dtau2)



1.0 0.3333333333333333 1.0 2.0
0.8836969710179877 0.03680440423587853 0.35402744418159326 0.012325860932660805 0.9335290041768233 0.030004341575321107 1.4961256126932017 0.11233444305653939
0.9651271402621416 0.042672062254350815 0.34475390465257616 0.012191846051549191 0.9940393362261984 0.03301872448410946 1.7994668870675827 0.1299252548822552
1.0207488391283706 0.047373009213783616 0.33364610832719194 0.01201032378683758 1.0220374415941822 0.03490565793576527 2.0593758286170907 0.14536817654404155
0.9449397430530311 0.04134250150218369 0.3300995773388022 0.011431445968558617 0.9824045878976163 0.03255934557106451 1.862589981698702 0.13084404351472986
0.8844818361675469 0.03699920133530459 0.35251734124435846 0.012416513312756668 1.026674476369725 0.0337139885811952 1.5090448970407973 0.11449822545998187
0.9292041055421261 0.04028258131035407 0.3243034982231066 0.01121549295155174 0.988659676466965 0.032809104882049196 1.8652299794277098 0.13020766441683554
1.0175870868206291 0.04710

1.0564833105721538 0.050852932694860124 0.3221793265011944 0.011657315608561609 1.0049661472844458 0.03484876051230194 2.279177848080321 0.1573871642636022
0.9742061811077734 0.04338906596076698 0.35062998408681173 0.012599805885482275 1.0552556818618046 0.035529012919274 1.77844515677978 0.1310248358176093
1.07187506506323 0.052263733660590554 0.32452120932455214 0.0116069887891069 1.0021567351834086 0.034772737496354886 2.302943025801598 0.15942394522748687
1.0587730427281323 0.050879553227165863 0.3157254969487157 0.011160544035333555 1.0231609845353942 0.03555170713152578 2.3534606896196038 0.16033864082895335
0.9788646830490311 0.04376319368177156 0.3439660217008197 0.01234484869095823 1.0247442667828763 0.03444063750508306 1.8458179625092266 0.13352789524022354
0.9596129808904976 0.042492942054800246 0.3159349703147929 0.011170733514835456 0.9819221718873163 0.03297870861243373 2.03737500136295 0.13926715655616662
1.1051807820403678 0.055583772077430274 0.32033952664238585 0.0113

1.0294136917897323 0.04808492446653664 0.3349272222277332 0.011969205101256 1.0288201729204989 0.03519003341033487 2.073544410462355 0.1464737093103673
0.9498940658391662 0.04167468046397968 0.3376031693261023 0.012059857309135178 1.0218030333832397 0.034195562385380275 1.8136408426949084 0.13072339445801404
1.1414885593293989 0.060025966990605376 0.32917652590581 0.01189250942219471 0.9898357170787025 0.03528903809454277 2.4677094795514747 0.17370610143848414
0.9372776404845756 0.04056199939120449 0.3408565493450656 0.012041504447353704 0.954126472240983 0.03132007097536406 1.749771545494712 0.1257198680679831
0.8830421340780172 0.03686826293755705 0.34639614627617954 0.01219697835250793 0.9995215669791291 0.03271543314106384 1.5492262069623983 0.1155933697638554
0.8968018530029375 0.03781614445279343 0.35012072308167713 0.012319906911605958 1.0249698497146327 0.033735889129895384 1.5614075199819837 0.11708139222942081
0.8922339348413288 0.037519650764080925 0.334837109261953 0.011927

0.9989299085347437 0.04562605280205908 0.319488840386412 0.011476631329649697 0.9856778746541357 0.03346634048717459 2.126650393567952 0.14593071658898962
0.9019586076814241 0.0384324519674077 0.3119002277940837 0.010964281532137665 0.9906311178219976 0.03303534269958158 1.8918177266510254 0.13044413986337694
0.9772406921655596 0.043514581834025075 0.3582360035993373 0.01276232934954372 0.9941376349006302 0.032951717360552225 1.7279242799351267 0.12785110033797875
1.0102219138272965 0.04673927202427089 0.306372534284432 0.010976127018236713 1.0711922092543342 0.0373752118512811 2.297364485321065 0.15595126051419422
0.9417007086199944 0.04118292211736348 0.3180467454815071 0.011260315295309993 1.0152713742355113 0.03419298133837411 1.960887737412015 0.13595827249194253
0.9349764281223687 0.04030472266083421 0.3677636093260878 0.012666366132967745 0.9311431926503202 0.029961810719931273 1.5423299217551072 0.11640938062571554
1.1828405677532692 0.06502815928545243 0.324719481566534 0.0117

0.9181181487868152 0.039388456210902874 0.33934566377275727 0.012049528909804708 1.057989341669974 0.035400714191859244 1.7055543853998756 0.12486563836753453
1.0707048217382338 0.051680156759852186 0.37379245033293446 0.013279461945310364 0.9750051848128578 0.03284027074605715 1.8644367236003943 0.1399433358918205
0.9095595744175318 0.03892565735434834 0.30983208007932006 0.011054947315459043 1.0127719800515453 0.03407635399710191 1.9356533196455177 0.1333543861064405
0.9987779481975043 0.04523558347170733 0.35693949028146976 0.012412442205852553 0.931099954578386 0.030551654259610005 1.7981716100112757 0.1306458077021253
0.9794069978699128 0.044052885077067665 0.3131611097406385 0.011038723973502307 0.9690163511696208 0.032637067708324065 2.1274860364400365 0.1438183829357476
0.9779428048542642 0.04374270302841699 0.33550487282492847 0.01186844241553439 0.9828804295337309 0.03281902802181826 1.9148393482933692 0.13530214257032286
1.0621708327018664 0.050653116391269244 0.376749100617

In [21]:
corr1k05['A1'] = A1_list
corr1k05['A2'] = A2_list
corr1k05['dA1'] = dA1_list
corr1k05['dA2'] = dA2_list
corr1k05['D'] = D_list
corr1k05['dD'] = dD_list
corr1k05['C'] = C_list
corr1k05['dC'] = dC_list
corr1k05['tau1'] = tau1_list
corr1k05['tau2'] = tau2_list
corr1k05['dtau1'] = dtau1_list
corr1k05['dtau2'] = dtau2_list

In [22]:
corr1k05

,rho,coupling,pearson,a1,da1,a2,da2,d,dd,a1ep,...,dA1,dA2,D,dD,C,dC,tau1,tau2,dtau1,dtau2
0,0.5,2.0,0.429276,0.900333,0.077475,0.356596,0.020181,0.932768,0.038209,1.374045,...,0.036804,0.012326,0.933529,0.030004,1.496126,0.112334,0.946620,0.379236,0.269081,0.270045
1,0.5,2.0,0.475215,0.983414,0.083328,0.347273,0.018769,0.993904,0.042320,1.093765,...,0.042672,0.012192,0.994039,0.033019,1.799467,0.129925,0.970914,0.346821,0.271775,0.273005
2,0.5,2.0,0.501532,1.041411,0.091089,0.335957,0.017564,1.025705,0.042548,1.451544,...,0.047373,0.012010,1.022037,0.034906,2.059376,0.145368,0.998739,0.326452,0.278759,0.280232
3,0.5,2.0,0.476623,0.962169,0.081413,0.332260,0.017791,0.983599,0.040760,1.229173,...,0.041343,0.011431,0.982405,0.032559,1.862590,0.130844,0.961864,0.336012,0.270662,0.271861
4,0.5,2.0,0.431407,0.899511,0.072164,0.354984,0.019200,1.024875,0.043741,2.397144,...,0.036999,0.012417,1.026674,0.033714,1.509045,0.114498,0.861502,0.343358,0.241376,0.242300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0.5,2.0,0.469117,0.965383,0.082081,0.343594,0.019369,0.961891,0.040275,3.816385,...,0.041238,0.011988,0.965788,0.031797,1.769707,0.127191,0.978795,0.353393,0.275792,0.276973
496,0.5,2.0,0.481844,0.922816,0.078574,0.319107,0.017319,0.964434,0.039848,1.082935,...,0.038555,0.011102,0.964869,0.031926,1.859651,0.128753,0.938106,0.328049,0.266220,0.267321
497,0.5,2.0,0.514214,1.040725,0.090418,0.316965,0.016064,1.022044,0.042236,1.210762,...,0.047524,0.011159,1.018963,0.035038,2.244037,0.152792,1.002003,0.308875,0.280457,0.281981
498,0.5,2.0,0.445304,1.053270,0.093172,0.393661,0.021849,0.969351,0.039463,4.406811,...,0.047736,0.013585,0.978079,0.032186,1.639512,0.126901,1.053819,0.399248,0.293658,0.295064


In [24]:
# display statistics
print(corr1k05['A1'].mean(),corr1k05['A1'].std(),corr1k05['dA1'].mean(),corr1k05['dA1'].std())
print(corr1k05['a1'].mean(),corr1k05['a1'].std(),corr1k05['da1'].mean(),corr1k05['da1'].std())
print(corr1k05['A2'].mean(),corr1k05['A2'].std(),corr1k05['dA2'].mean(),corr1k05['dA2'].std())
print(corr1k05['a2'].mean(),corr1k05['a2'].std(),corr1k05['da2'].mean(),corr1k05['da2'].std())

1.0002765955058968 0.08133195432362353 0.04624707377453065 0.007171679890537052
1.0208463041012512 0.08486960869000124 0.0881176131435236 0.011364509943173657
0.3349989811398769 0.01802209288275044 0.011874967264527405 0.0006261756380823517
0.33722173430594266 0.01826424965060615 0.017910720127188764 0.0013345058206388914


In [25]:
corr10k05 = pd.read_csv("correlations10k05.csv")
corr10k05

,rho,coupling,pearson,a1,da1,a2,da2,d,dd,a1ep,a1ss,a1c,a2ep,a2ss,a2c,c,dc
0,0.5,2.0,0.496261,1.000833,0.026690,0.335132,0.005588,1.009502,0.013097,1.328084,9981.792163,7407.956161,1.469984,3359.226426,1330.012663,1.987214,0.093895
1,0.5,2.0,0.501621,1.009224,0.027040,0.333549,0.005511,1.011162,0.013052,3.098745,10061.246461,7472.013107,1.049581,3342.642595,1321.126376,2.026493,0.093905
2,0.5,2.0,0.487703,0.973675,0.024953,0.334037,0.005633,0.981533,0.012751,1.000398,9713.627178,7190.444794,2.700469,3340.335444,1369.068701,1.915713,0.089913
3,0.5,2.0,0.492071,0.981422,0.025726,0.333887,0.005387,0.984915,0.012820,1.138366,9794.038011,7239.572834,1.184801,3334.176612,1379.951776,1.940144,0.090461
4,0.5,2.0,0.508697,1.003508,0.026250,0.326404,0.005527,0.983765,0.012471,1.478120,10022.640270,7451.525742,1.004958,3254.743123,1329.592121,2.075335,0.096485
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0.5,2.0,0.493179,0.984850,0.026441,0.334191,0.005535,1.000931,0.012829,1.120238,9826.222694,7235.515953,1.932336,3335.364339,1360.933757,1.947788,0.093221
496,0.5,2.0,0.508138,1.027355,0.027098,0.333789,0.005540,1.007071,0.012714,1.166181,10244.600998,7641.608912,1.074860,3338.409800,1338.382690,2.078706,0.096065
497,0.5,2.0,0.484874,0.985330,0.025065,0.339212,0.005620,0.991186,0.012881,1.654235,9821.831612,7301.138928,1.780081,3401.862254,1379.637614,1.905529,0.087185
498,0.5,2.0,0.494891,1.024356,0.027848,0.343992,0.005859,0.991815,0.012954,1.918978,10212.802785,7662.972111,1.009481,3447.216153,1426.402453,1.978727,0.096141


In [26]:
rho = 0.5
delta_t = 0.3
N = 10000
c = 2*rho/(1-rho)
guessa1 = 1.0
guessa2 = 1.0/(1.0+c)
guessd = 1.0
A1_list = []
A2_list = []
dA1_list = []
dA2_list = []
C_list = []
dC_list = []
D_list = []
dD_list = []
tau1_list = []
dtau1_list = []
tau2_list = []
dtau2_list = []
print(guessa1,guessa2,guessd,c)
for index, row in corr10k05.iterrows():
    a1ep,a1ss,a1c = row['a1ep'], row['a1ss'], row['a1c']
    a2ep,a2ss,a2c = row['a2ep'], row['a2ss'], row['a2c']
    para = (a1ep,a1ss,a1c,a2ep,a2ss,a2c,delta_t,N)
    result = root(phi_deriv, [guessa1,guessa2,guessd],args=para)
    A1 = result.x[0]
    A2 = result.x[1]
    D = result.x[2]
    # lets calculate the Hessian
    b1 = b(D,A1,delta_t)
    b2 = b(D,A2,delta_t)
    d2PdA2_1m = d2PdA2(N,a1ep,a1ss,a1c,b1,D,A1,delta_t)
    d2PdA2_2m = d2PdA2(N,a2ep,a2ss,a2c,b2,D,A2,delta_t)
    d2PdD2m = d2PdD2(N,a1ep,a1ss,a1c,a2ep,a2ss,a2c,b1,b2,D,A1,A2,delta_t)
    d2PdAdD_1m = d2PdAdD(N,a1ep,a1ss,a1c,b1,D,A1,delta_t)
    d2PdAdD_2m = d2PdAdD(N,a2ep,a2ss,a2c,b2,D,A2,delta_t)
    hessian = np.array([[d2PdA2_1m,0,d2PdAdD_1m],[0,d2PdA2_2m,d2PdAdD_2m],[d2PdAdD_1m,d2PdAdD_2m,d2PdD2m]])
    var = -np.linalg.inv(hessian)
    dA1 = np.sqrt(var[0,0])
    dA2 = np.sqrt(var[1,1])
    dD = np.sqrt(var[2,2])
    dA1A2 = var[0,1]
    dA1dD = var[0,2]
    dA2dD = var[1,2]
    C = (A1-A2)/A2
    dC = np.sqrt(1/A2**2*dA1**2+A1**2/A2**4*dA2**2-A1/A2**4*dA1A2)
    tau1 = A1/D
    tau2 = A2/D
    dtau1 = np.sqrt(1/D**2*dA1+A1**2/D**4*dD-A1/D**3*dA1dD)
    dtau2 = np.sqrt(1/D**2*dA1+A1**2/D**4*dD-A1/D**3*dA2dD)
    print(A1,dA1,A2,dA2,D,dD,C,dC)
    # add results to list
    A1_list.append(A1)
    A2_list.append(A2)
    dA1_list.append(dA1)
    dA2_list.append(dA2)
    D_list.append(D)
    dD_list.append(dD)
    C_list.append(C)
    dC_list.append(dC)
    tau1_list.append(tau1)
    tau2_list.append(tau2)
    dtau1_list.append(dtau1)
    dtau2_list.append(dtau2)



1.0 0.3333333333333333 1.0 2.0
0.9994675419572917 0.014391687200710768 0.3349046017622491 0.00372937859275173 1.009193155082451 0.010756545984367516 1.9843350515285543 0.04424014208940388
1.007546403997519 0.014609603913641288 0.3334404277154273 0.0037209450866414256 1.0117570214559755 0.010823076870172529 2.021668400861708 0.0449392650566639
0.9719008475147249 0.0136923632286232 0.3338177108839614 0.003710077715096684 0.9815972584171282 0.010342858131802994 1.9114717878242478 0.04257181346477782
0.9792518604340514 0.013874532549799982 0.33370636296150985 0.0037234409134307305 0.9847244658845904 0.01040642686321342 1.9344716467003649 0.0430541434471078
1.001822968698728 0.01447850085533472 0.32608144176562875 0.0036531446989494813 0.9837547510495135 0.01049989364268281 2.072309062650639 0.04534873397942453
0.9873808756570053 0.014088338078897134 0.3329772011373687 0.0037227890989870306 1.0128511096058317 0.01079036838713824 1.9653107548635576 0.043797865274509004
1.0419130848177882 0.0

0.9924234948104929 0.014224729120517343 0.3284248347921471 0.003652650927837627 0.9822231510585598 0.01042931763757029 2.021767508655598 0.044426826766552616
0.982873565439927 0.01396804998066855 0.33367914235557977 0.003705863764142985 0.9875297224520904 0.010442485466217238 1.9455648875785698 0.043256244946661276
0.9996535116617127 0.014394577749555344 0.3349643983181981 0.003735041779770316 0.9903619581032008 0.010520871710528509 1.9843574919627605 0.04414567238882648
1.0486900507559012 0.015769921839125168 0.3322607670480711 0.0037304654512907505 0.9934169499427588 0.01073478534622654 2.1562259368533208 0.047628884610339046
1.0098964496741425 0.01465176704695756 0.340078306622671 0.003794172053425453 1.0105396266969546 0.010777323401483248 1.9695997363179607 0.044279733959560835
0.9911803604087513 0.014178202567749907 0.33539726499349914 0.0037330943751321647 1.0019420967050814 0.010639810297614674 1.9552428235452752 0.04364951888993506
1.0121508338034948 0.01470817778415304 0.3430

1.0085039135866312 0.014605936027464942 0.3443330676264657 0.003852800243237033 1.002218853809746 0.010652660907235899 1.9288616412544537 0.04369089500542418
1.0394321918806653 0.015487487485356149 0.335004893786265 0.003750834244100727 0.9934962261618003 0.010683906715939755 2.102737336559115 0.04666138029562497
1.0096235834985219 0.01465680305338324 0.33625701533573754 0.003762845445057047 0.9958030096402902 0.010616829638959213 2.0025353745867815 0.044654281991643965
1.0170453697089183 0.014878145693311973 0.3301490189820292 0.003706545792612681 0.9908646037608746 0.01061506775769867 2.0805645670092954 0.04584998298927206
1.005702254251814 0.014557908788528579 0.33611165843553276 0.0037826681210584067 1.022400136858519 0.010956419790519555 1.9921671236664664 0.044632496118119584
0.9684205330518314 0.013600275041505766 0.3381022077111442 0.0037409038388094736 0.987562371431643 0.0103803326330737 1.864283376342802 0.041889592868902495
1.0147469572422112 0.01479132828096024 0.336211249

0.9686999059615911 0.013635803933337015 0.32949397004868825 0.0036837890699897478 1.0215596906466176 0.010876216495020801 1.9399624697788835 0.04316196505837328
0.9767566756907285 0.01380007009626381 0.3404275535175104 0.003792470651462839 1.0053060055052891 0.010622279372169472 1.8692056961848935 0.04224977038009868
0.9549935839255707 0.013295221338586438 0.33129027780630305 0.003693221028574248 1.0005601936663562 0.010562881801657837 1.8826489876166272 0.04201642367314222
1.0372891323482896 0.015399949962329027 0.34270407754253907 0.0038271777030469117 0.9890675583309723 0.010574809842517869 2.026777912263192 0.04554659448917255
0.996628190179469 0.014330631579560833 0.3317259999680168 0.0037070834173859256 1.0133769377258264 0.010827071869206152 2.0043716509274474 0.044507862997105914
1.0479862386873762 0.015735831812947465 0.33664134024171666 0.003780058180441572 0.9959705047556908 0.010741315292975208 2.1130645984681995 0.0470491010859894
0.9823699148138699 0.013951844403764842 0.

0.951565835736636 0.013214222799685899 0.32758443307863905 0.003665106151628671 0.9881541816869513 0.010423535929776592 1.9047956485411064 0.04219545526736111
0.9685479439735596 0.013641162152627413 0.32545147691026927 0.003638577031917958 1.013167616799705 0.010791749619483931 1.976013362018325 0.04359096172975061
1.0176571675539798 0.014886410299348314 0.332553207849741 0.0037307361269977093 1.0150069768564352 0.010909353169657415 2.06013336672967 0.04572972913603636
0.9936384627311031 0.014268266441583194 0.32559625148248067 0.003649426697237176 1.0025506325536124 0.010721225378512271 2.051750314098956 0.04502849517381913
0.9873524934794872 0.014082712747088378 0.3337464214489061 0.0037238729338276272 0.9997574619962879 0.01061601029344136 1.9583912516366648 0.04361458351829166
0.9579939189201734 0.013377951337927513 0.32615281849678457 0.003637819227553635 0.9947376313707191 0.010524312956707561 1.937254760929248 0.04275527591572986
1.0355967160755686 0.015355634786236708 0.3396460

In [27]:
corr10k05['A1'] = A1_list
corr10k05['A2'] = A2_list
corr10k05['dA1'] = dA1_list
corr10k05['dA2'] = dA2_list
corr10k05['D'] = D_list
corr10k05['dD'] = dD_list
corr10k05['C'] = C_list
corr10k05['dC'] = dC_list
corr10k05['tau1'] = tau1_list
corr10k05['tau2'] = tau2_list
corr10k05['dtau1'] = dtau1_list
corr10k05['dtau2'] = dtau2_list

In [28]:
# display statistics
print(corr10k05['A1'].mean(),corr10k05['A1'].std(),corr10k05['dA1'].mean(),corr10k05['dA1'].std())
print(corr10k05['a1'].mean(),corr10k05['a1'].std(),corr10k05['da1'].mean(),corr10k05['da1'].std())
print(corr10k05['A2'].mean(),corr10k05['A2'].std(),corr10k05['dA2'].mean(),corr10k05['dA2'].std())
print(corr10k05['a2'].mean(),corr10k05['a2'].std(),corr10k05['da2'].mean(),corr10k05['da2'].std())

0.9994679035701323 0.026845608796935903 0.014417519509535145 0.0007050718168306769
1.0014265763170065 0.026974669313677008 0.02632549546333618 0.001154753441332831
0.333678731101721 0.005615460030842642 0.003732587851557967 6.095148736597161e-05
0.33389375664610776 0.005620064502636682 0.005541406537549055 0.00015467315469345932
